# Transportation Networks

Keywords: transportation, assignment, cbc usage

Tutorial ini diambil dari contoh penggunaan pyomo pada link berikut ini:
https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/03.01-Transportation-Networks.html#

Model transportation network problems menggunakan Pyomo dan GLPK. Deskrispi masalah dari Johannes Bisschop, ["AIMMS Optimization Modeling", AIMMS B. V., 2014](http://download.aimms.com/aimms/download/manuals/AIMMS3_OM.pdf).


## Latar Belakang Permasalahan

Permasalahan transportasi berkaitan dengan distributisi komoditas dari sejumlah titik sumber pasokan ke titik-titik tujuan. Tujuannya adalah meminimasi total biaya transportasi dan memenuhi sejumlah batasan pasokan untuk setiap sumber pemasok, dan memenuhi permintaan (demand) sejumlah titik.

Terdapat dua pabrik dan enam pelanggan yang berada di delapan lokasi kota-kota di Eropa seperti diilustrasikan dalam peta. Pelanggan berwarna merah dan pabrik berwarna biru.

![images/TransportationNetworksMap.png](images/TransportationNetworksMap.png)

**Biaya transportasi** antara titik pabrik ke pelanggan **per unit** dalam **&euro;/ton** pengiriman barang, disajikan dalam tabel.

*Analogi kasus lain*.

Biaya alokasi adalah biaya (rp) perkilometer dikali dengan demand potensial yang dilayani oleh titik sumber. Sehingga **Biaya (Rp) = (Cij)/demand x demand = (Rp/km x km)/demand x demand**.  Demand di suatu titik dalam bentuk potensial demand (berupa bobot permintaan).  
sehingga berapa alokasi ke titik permintaan untuk melayani sebagian atau seluruh potensial demand, dipengaruhi juga jarak km dari titik permintaan ke titik pelayanan (sources).

Tujuannya sama, meminimumkan biaya, karena jika jarak km dari titik permintaan ke titik layanan jauh, maka Cij besar dikali dengan besarnya potensial demand yang akan dilayani.




### Table Biaya transportasi, demand pelanggan, dan kapasitas suplai pemasok

| Customer\Pemasok | Arnhem [&euro;/ton] | Gouda [&euro;/ton] | Demand [tons]|
| :--: | :----: | :---: | :----: |
| London | - | 2.5 | 125 |
| Berlin | 2.5 | - | 175 |
| Maastricht | 1.6 | 2.0 | 225 |
| Amsterdam | 1.4 | 1.0 | 250 |
| Utrecht | 0.8 | 1.0 | 225 |
| The Hague | 1.4 | 0.8 | 200 |
| **Supply [tons]** | 550 tons | 700 tons |  |

#### Table biaya untuk analogi kasus yang lain di atas.
Nilai di bawah kolom Jebres dan Manahan adalah menyatakan Biaya (Cij)/demand), seperti dijelaskan sebelumnya. Misalkan Jebres ke Manahan Rp 7/demand. Baris supply adalah informasi maksimum nilai supply yg bisa dilakukan dari sumber layanan.
| Permintaan\Sumber | Jebres  [Rp/demand] | Manahan [Rp/demand] | Demand [bobot]|
| :--: | :----: | :---: | :----: |
| Jebres | - | 7 | 125 |
| Manahan | 7.5 | - | 175 |
| .... | - | - | 225 |
| Balapan | 4 | 4.5 | 250 |
| Purwosari | 9 | 4.5 | 225 |
| **Supply [demand]** | 600  | 650 |  |


Situasi masalah tersebut dapat dimodelkan sebagai model jaringan transportasi antara titik sumber (pemasok) ke titik pelanggan.

![TransportNet.png](images/TransportNet.png)

Untuk setiap titik terdapat sebuah **parameter** $T[c,s]$ menyatakan biaya pengiriman per ton barang melalui sebuah link. Permasalahannya adalah menentukan jumlah barang yang dikirim pada setiap link, sebagai **variabel keputusan** non-negative $x[c,s]$.

**Fungsi tujuan** adalah meminimasi total  biaya pengiriman ke semua titik pelanggan dari titik pemasok. 

$${minimize:}\quad {Cost} = \sum_{c \in Pelanggan}\sum_{s \in Pemasoks} T[c,s] x[c,s]$$

Pengiriman dari semua titik pemasok tidak boleh melebihi kapasitas produksi titik sumber.

$$\sum_{c \in Pelanggan} x[c,s] \leq {Supply}[s] \qquad \forall s \in Pemasok$$

Pengiriman ke titik pelanggan harus memenuhi demand setiap pelanggan.

$$\sum_{s\in Pemasoks} x[c,s] = {Demand}[c] \qquad \forall c \in Pelanggan$$

## Pyomo model

In [1]:
from pyomo.environ import *

### Data File

In [2]:
# Tipe data dictionary
Demand = {
   'Lon': 125,        # London
   'Ber': 175,        # Berlin
   'Maa': 225,        # Maastricht
   'Ams': 250,        # Amsterdam
   'Utr': 225,        # Utrecht
   'Hag': 200         # The Hague
}

Supply = {
   'Arn': 600,        # Arnhem
   'Gou': 650         # Gouda
}

T = {
    ('Lon', 'Arn'): 1000,
    ('Lon', 'Gou'): 2.5,
    ('Ber', 'Arn'): 2.5,
    ('Ber', 'Gou'): 1000,
    ('Maa', 'Arn'): 1.6,
    ('Maa', 'Gou'): 2.0,
    ('Ams', 'Arn'): 1.4,
    ('Ams', 'Gou'): 1.0,
    ('Utr', 'Arn'): 0.8,
    ('Utr', 'Gou'): 1.0,
    ('Hag', 'Arn'): 1.4,
    ('Hag', 'Gou'): 0.8
}

### Model file

In [3]:
# Step 0: Membuat sebuah instance model --> instance adalah representasi sebuah objek nyata dari konsep OOP
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)
# Manual ttg Suffix : 
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Suffixes.html#importing-suffix-data
# dual adalah nama tambahan fungsi yang diberikan oleh solver, tidak semua solver mengakomodir

# Step 1: Define index sets
CUS = list(Demand.keys())
SRC = list(Supply.keys())

# Step 2: Define the decision, variabel keputusan 
model.x = Var(CUS, SRC, domain = NonNegativeReals)

# Step 3: Define Objective
@model.Objective(sense=minimize)
def cost(m):
    return sum([T[c,s]*model.x[c,s] for c in CUS for s in SRC]) 
#perhatikan for c in CUS dst dan lihat model sigma pada rumusan matematik

# Step 4: Constraints
# alokasi suplai tidak melebihi kapasitas sumber
@model.Constraint(SRC)
def src(m, s):
    return sum([model.x[c,s] for c in CUS]) <= Supply[s]

# Demand setiap titik terpenuhi
@model.Constraint(CUS)
def dmd(m, c):
    return sum([model.x[c,s] for s in SRC]) == Demand[c]
    
results = SolverFactory('glpk').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1705.0
  Upper bound: 1705.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 12
  Number of nonzeros: 24
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0672609806060791
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

## Solution

In [4]:
model.x.display()

x : Size=12, Index=x_index
    Key            : Lower : Value : Upper : Fixed : Stale : Domain
    ('Ams', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Ams', 'Gou') :     0 : 250.0 :  None : False : False : NonNegativeReals
    ('Ber', 'Arn') :     0 : 175.0 :  None : False : False : NonNegativeReals
    ('Ber', 'Gou') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Hag', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Hag', 'Gou') :     0 : 200.0 :  None : False : False : NonNegativeReals
    ('Lon', 'Arn') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Lon', 'Gou') :     0 : 125.0 :  None : False : False : NonNegativeReals
    ('Maa', 'Arn') :     0 : 225.0 :  None : False : False : NonNegativeReals
    ('Maa', 'Gou') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('Utr', 'Arn') :     0 : 200.0 :  None : False : False : NonNegativeReals
    ('Utr', 'Gou') :     0 :  25.0 :  None : Fa

In [5]:
for c in CUS:
    for s in SRC:
        print(c, s, model.x[c,s]())

Lon Arn 0.0
Lon Gou 125.0
Ber Arn 175.0
Ber Gou 0.0
Maa Arn 225.0
Maa Gou 0.0
Ams Arn 0.0
Ams Gou 250.0
Utr Arn 200.0
Utr Gou 25.0
Hag Arn 0.0
Hag Gou 200.0


In [6]:
if 'ok' == str(results.Solver.status):
    print("Total Biaya Pengiriman = ", model.cost())
    print("\nTabel Kuantitas Pengiriman:")
    for s in SRC:
        for c in CUS:
            if model.x[c,s]() > 0:
                print("Pengiriman dari ", s," ke ", c, ":", model.x[c,s]())
else:
    print("No Valid Solution Found")

Total Biaya Pengiriman =  1705.0

Tabel Kuantitas Pengiriman:
Pengiriman dari  Arn  ke  Ber : 175.0
Pengiriman dari  Arn  ke  Maa : 225.0
Pengiriman dari  Arn  ke  Utr : 200.0
Pengiriman dari  Gou  ke  Lon : 125.0
Pengiriman dari  Gou  ke  Ams : 250.0
Pengiriman dari  Gou  ke  Utr : 25.0
Pengiriman dari  Gou  ke  Hag : 200.0


Selain Utrecht, pelanggan lain hanya dilayani oleh satu pemasok.

![TransportNet_soln.png](images/TransportNet_soln.png)


## Sensitivity analysis

### Analisis pada Sumber Pasokan

In [7]:
if 'ok' == str(results.Solver.status):
    print("Source      Capacity   Shipped    Margin")
    for s in SRC:
        print(f"{s:10s}{Supply[s]:10.1f}{model.src[s]():10.1f}{model.dual[model.src[s]]:10.4f}")
else:
    print("No Valid Solution Found")

Source      Capacity   Shipped    Margin
Arn            600.0     600.0   -0.2000
Gou            650.0     600.0    0.0000


Nilai 'Margin' menginformasikan berapa total biaya yang akan ditingkatkan untuk setiap kenaikan satu ton pasokan yang tersedia dari masing-masing sumber. Perhitungan optimasi mengatakan bahwa hanya 650 ton dari 700 yang tersedia dari Gouda yang harus digunakan untuk solusi biaya minimum. Kapasitas Gouda bisa diturunkan dengan Nilai margin Gouda adalah 0.

Sumber di Arnhem berbeda. Pertama, semua 550 ton yang tersedia telah digunakan. Kedua, dari nilai margin dapat dilihat bahwa **total biaya transportasi akan berkurang sebesar 0,2 Euro** untuk setiap ton pasokan tambahan.

Informasi strategis bagi manajemen, ada kelebihan pasokan yang tersedia di Gouda yang seharusnya, jika memungkinkan, dipindah ke Arnhem.

### Analysis pada Customer

In [8]:
if 'ok' == str(results.Solver.status):    
    print("Customer      Demand   Shipped    Margin")
    for c in CUS:
        print(f"{c:10s}{Demand[c]:10.1f}{model.dmd[c]():10.1f}{model.dual[model.dmd[c]]:10.4f}")
else:
    print("No Valid Solution Found")

Customer      Demand   Shipped    Margin
Lon            125.0     125.0    2.5000
Ber            175.0     175.0    2.7000
Maa            225.0     225.0    1.8000
Ams            250.0     250.0    1.0000
Utr            225.0     225.0    1.0000
Hag            200.0     200.0    0.8000


Perhatikan pada kolom demand, semua demand dapat dipenuhi oleh solusi.

**Nilai Margin** pada batasan-batasan mengindikasikan *berapa besar total biaya ransportasi akan bertambah jika ada **penambahan** demand per ton*. Sebagai contoh, penambahan 1 ton demand di Berlin akan meningkatkan biaya sebesar 2.7 Euros. Penambahan biaya tersebut **lebih besar** dari biaya awal untuk pengiriman ke  Berlin sebesar 2.5 Euros per ton.  

Perhatikan ilustrasi gambar berikut ini.

![TransportNet_sens.png](images/TransportNet_sens.png)


* Jika permintaan Berlin meningkat dari 175 menjadi 176 tons, maka total biaya meningkat dari 437.5 menjadi 440.0, atau terjadi peningkatan biaya sebesar 2.5 Euros.
* Oleh karena pemasok Arnhem sudah beroperasi dengan kapasitas penuh, peningkatan pengiriman dari Arnhem ke Berlin akan mengurangi pengiriman dari Arhhem ke Utrecht dari 150 menjadi 149 ton sehingga terjadi pengurangan biaya kirim sebesar 120.0 menjadi 119.2, atau penurunan sebesar 0.8 Euros.
* Kemudian untuk memenuhi kebutuhan di Utrecht, pengiriman dari Gouda ke Utrecht harus ditambah dari 75 menjadi 76 ton, yang menambah biaya kirim sebesar 1.0 Euros.
* Dengan demikian, efek penambahan permintaan 1 ton pada Berlin akan menambah margin biaya kirim sebesar **2.5 - 0.8 + 1.0 = 2.7 Euros**.

Kesimpulan penting yang dapat diperoleh bahwa solusi optimal yang diberikan di atas pada kondisi normal dengan total permintaan yang telah diketahui (deterministik) sebelumnya. Jika ada *ripple effect* pada rantai pasok, maka solusi optimal dapat berubah sesuai hasil analisis sensitivitas di atas.

## Latihan

1. Pindahkan 50 ton kapasitas pasokan dari Gouda ke Arnhem, dan ulangi analisis sensitivitas. Bagaimana hasilnya? Dalam prakteknya, apakah Anda akan merekomendasikan solusi ini, ataukah Anda memiliki pendapat lain?
2. Apa rekoemndasi bisnis lain yang bisa Anda berikan?

In [9]:
# Ubah kapasitas di Arn (bertambah 50), dan Gouda dikurangi
Supply = {
   'Arn': 650,        # Arnhem
   'Gou': 600         # Gouda
}


In [14]:
# Step 0: Create an instance of the model
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)
# dual adalah penamaan fungsi yang diberikan untuk model yg dibuat

# Step 1: Define index sets
CUS = list(Demand.keys())
SRC = list(Supply.keys())

# Step 2: Define the decision, variabel keputusan 
model.x = Var(CUS, SRC, domain = NonNegativeReals)

# Step 3: Define Objective
@model.Objective(sense=minimize)
def cost(m):
    return sum([T[c,s]*model.x[c,s] for c in CUS for s in SRC])

# Step 4: Constraints
# alokasi suplai tidak melebihi kapasitas sumber
@model.Constraint(SRC)
def src(m, s):
    return sum([model.x[c,s] for c in CUS]) <= Supply[s]

# Demand setiap titik terpenuhi
@model.Constraint(CUS)
def dmd(m, c):
    return sum([model.x[c,s] for s in SRC]) == Demand[c]
    
results = SolverFactory('glpk').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1700.0
  Upper bound: 1700.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 12
  Number of nonzeros: 24
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0545656681060791
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [15]:
if 'ok' == str(results.Solver.status):
    print("Total Biaya Pengiriman = ", model.cost())
    print("\nTabel Pengiriman:")
    for s in SRC:
        for c in CUS:
            if model.x[c,s]() > 0:
                print("Pengiriman dari ", s," ke ", c, ":", model.x[c,s]())
else:
    print("No Valid Solution Found")

Total Biaya Pengiriman =  1700.0

Tabel Pengiriman:
Pengiriman dari  Arn  ke  Ber : 175.0
Pengiriman dari  Arn  ke  Maa : 225.0
Pengiriman dari  Arn  ke  Utr : 225.0
Pengiriman dari  Gou  ke  Lon : 125.0
Pengiriman dari  Gou  ke  Ams : 250.0
Pengiriman dari  Gou  ke  Hag : 200.0


In [16]:
# Analisis berbasiskan sumber
if 'ok' == str(results.Solver.status):
    print("Source      Capacity   Shipped    Margin")
    for s in SRC:
        print(f"{s:10s}{Supply[s]:10.1f}{model.src[s]():10.1f}{model.dual[model.src[s]]:10.4f}")
else:
    print("No Valid Solution Found")

Source      Capacity   Shipped    Margin
Arn            650.0     625.0    0.0000
Gou            600.0     575.0    0.0000


Maksimum shipped dari Arn sebenarnya 650, dan jika bertambahpun tidak akan menambah biaya karena kebuthan supply dari Arn hanya sampai 625 ton saja.

Demikian juga dengan Goudo.

In [17]:
# Analisis sensitivitas dari konsumen
if 'ok' == str(results.Solver.status):    
    print("Customer      Demand   Shipped    Margin")
    for c in CUS:
        print(f"{c:10s}{Demand[c]:10.1f}{model.dmd[c]():10.1f}{model.dual[model.dmd[c]]:10.4f}")
else:
    print("No Valid Solution Found")

Customer      Demand   Shipped    Margin
Lon            125.0     125.0    2.5000
Ber            175.0     175.0    2.5000
Maa            225.0     225.0    1.6000
Ams            250.0     250.0    1.0000
Utr            225.0     225.0    0.8000
Hag            200.0     200.0    0.8000


Berlin dan London memiliki biaya terbesar yang sama jika terjadi penambahan 1 ton demand